# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **
In classification problems, we aim to predict the class that a new data belongs to given the classes of previously seen data. In regression problems, we aim to predict a desired continuous variable based on available data. 

In this case, we aim to accurately identify which student will fail or pass a course given data regarding his/her socioeconomic status, class attendance, scores, etc. Therefore, this is a a classification problem.  

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time,sleep
from sklearn.metrics import f1_score

# Read studnt data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [12]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1]-1

# TODO: Calculate passing students
n_passed = sum(student_data.passed=='yes')

# TODO: Calculate failing students
n_failed = sum(student_data.passed=='no')

# TODO: Calculate graduation rate
grad_rate = (float(n_passed))/(float(n_passed)+float(n_failed))*100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [13]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [14]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))


Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [15]:
from sklearn.cross_validation import StratifiedShuffleSplit

num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

ratio_test_tot  = float(num_test)/float(X_all.shape[0])

sss = StratifiedShuffleSplit(y_all, 1, test_size=ratio_test_tot, random_state=42)

for train_index, test_index in sss:
    train_index = train_index
    test_index =  test_index
    
# TODO: Shuffle and split the dataset into the number of training and testing points above

X_train = X_all.ix[train_index].reset_index()
X_train = X_train.drop('index',axis=1)
X_test = X_all.ix[test_index].reset_index()
X_test = X_test.drop('index',axis=1)

y_train = y_all[train_index]
y_test = y_all[test_index]

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

I first conducted [exploratory data analysis (EDA)](http://vxy10.github.io/2016/06/10/si-EDA/) on the features and tested dimensionality reduction techniques like [MCA and tSNE](http://vxy10.github.io/2016/06/24/si-mca/). I tested several model candidates, and at the end decided to use the following 3 for this project. 

1. Logistic Regression: 
 - Logistic regression is a statistical technique of modeling a binomial outcome (0 or 1, in our case passing or failing a course) with one or more explanatory variables. In logistic regression, given class labels \\(y\\) and a corresponding feature set \\(X\\), we aim to find a weight matrix \\(W\\) such that the difference between the class labels and \\( sig( WX) \\) is minimized. \\(sig\\) is the sigmoid function and the product \\( WX \\) is also refered as activation. We then interpret \\( sig( WX) \\) as class probabilities and threshold it (usually at 0.5) to indicate probability of the feature set belonging to a particular class. Logistic regression assumes that there is a single decision boundary and the data set is linearly separable and is less prone to overfitting. Logistic regression has been applied to predict outcomes in several real-world problems. For example, logistic regression was applied to predict [bankruptcy in the hospitality industry](http://scholarworks.umass.edu/cgi/viewcontent.cgi?article=1152&context=jhfm) using the financial data of 16 U.S. hospitality firms that went bankrupt between 1999 and 2004 and 16 non-bankrupt matching firms.  The logistic regression models could correctly predict 91% and 84% of bankruptcy cases 1 and 2 years earlier, respectively. 
 
 - Advantage: One advantage of using logistic regression is that in addition to providing predictions, it also gives how certain the model is of that prediction. This helps identify regions where the model is not sure of the prediction. Other advantage is that the predictor variables can be dichotomous, ordinal, or continuous. Another advantage is that the logistic regression provides a quantitative estimate of the strength of the association adjusting for other variables (removes confounding effects). The exponential of coefficients correspond to odd ratios for the given factor, therefore relative importance of each feature can be estimated. Logistic regression works better if there is a single decision boundary. Further,as logistic regression assumes a simple underlying model, it is less prone to overfitting.

 - Disadvantage: In cases where there are several independent factors or predictor/explanatory variables, enough data with each possible combination of predictor/explanatory is required. Using interaction or adding factors that are rare can result in overfitting, and degrade model's generalizability. Logistic regression assumes that there is a single decision boundary and the data set is linearly separable, however, this need not be true, and this assumption can result in poor classification. Further, although logistic regression allows for using ordinal data, in many applications the underlying linearity assumption between the ordinal predictors and outcome measure need not be valid. If we use coefficients of linear regression to draw inferences about relative importance of each feature, then some sort of feature scaling is required so the features are of the same order. This introduces additional preprocessing step and can adversely affect the scalability of the algorithm. 

 - Why use this model:  Logistic Regression is a good candidate for this data set because it is simple and is less prone to overfitting. Further, in addition to providing class predictions, it gives probability scores that are interpretted as faith of the model in the prediction. It also gives relative importance of each predictor. One limitation is lack of sufficient data. There are a total of 48 predictor variables, and only 395 training points. Even if we assume 2 levels for each of the 48 variables, we get a total \\(2^48\\) combinations, therefore there is not sufficient data to account for all possible combinations of data.

2. Decision tree learning:
 - Decision tree: A decision tree is a tree-like graph or model of decisions and their possible consequences, including chance event outcomes, resource costs, and utility. Decision tree learning uses a decision tree as a predictive model which maps features or predictor/explanatory variables to outcome variable. Decision tree learning partitions the feature space into different regions and assigns a class to each of the regions. A typical decision tree is implemented in two steps, first a 'best' split is obtained. This is typically achieved by splitting the data by individual feature and testing how well that feature alone explains the data. Most common splitting criteria are gini impurity which minimizes misclassification rate, entropy (or information gain) which splits data so the uncertainity in class predictions in each tree reduces.  The feature that shows highest predictive power is chosen to split upon. This process is repeated on the resulting subtrees and continued until there is no data left to split or some stopping criteria is met. Decision trees are very versatile and easy to use, and hence have found applications in several areas. For example, decision trees were used to build a [stock portfolio optimization](https://cs.uwaterloo.ca/~s255khan/files/finance_ICISTM_final_2007.pdf) algorithm.
  - Advantage: As decision trees make no underlying assumptions about the nature of data or its distribution, it is not prone to errors due to these assumptions. Therefore nonlinearities or skewed data distributions can easily be handled by decision trees. As the splits of features is based on minimizing misclassifiaction rates or maximizing information gain, decision trees inherently screen and select variables that are important for the classification problem. Therefore, features that do not provide any information or help in classifying data may never be used for prediction. More importantly, as data is split based on the values of individual features taken one at a time, there is no need to preprocess data, therefore decision trees are easy to scale up. As decision trees do not require specialized math or complex algorithms, they are easy to implement, understand and explain.  
 - Disadvantage: The main disadvantage of decision trees is that without proper pruning or limiting tree growth, they can easily overfit the training data. As trees are split based on one feature, they tend to ignore other correlated features, where as other algorithms will weigh in both the correlated features. Decision trees are very sensitive to data, and a small variation in data can result in a very different decision tree. Further, there are no probabilistic or statistical techniques (F-test, p-value, etc) that quantify how good the model fit is. A minor disadvantage is that decision trees assume that decision boundaries are functions of one variable alone, this results in decision boundaries that are parallel to axis of at least one feature. 
 - Why use this model:  Decision trees are insensitive to nonlinearity and skewness, both of which are expected to be present in our data set. Decision tress inherently split data based on features that provide the most information, therefore, features that are do not provide information are not used. Decision trees are easy to implement and relatively fast to compute, therefore, best parameter search over a wider grid can be performed. 

3. Adaboost:
 - Adaboost: AdaBoost or adaptive boosting is a meta-algorithm that can be used in conjunction with weak learners to improve their performance. Weighted sum of predictions from multiple weak algorithms is used to make predictions. The weights corresponding to individual weak learner are adapted in each iteration so the models that make greater errors contribute more to the overall error. Therefore, in the next iteration the algorithm improved the performances in areas where more errors are made. Repeating this step successively results in an algorithm that combines predictions from several weak classifiers to give a strong classifier. 
 - Advantage: As adaboost combines multiple weak learners, it is simple and easy to implement. It is flexible as it can be combined with any weak learner without requiring any knowledge or information about the weak learner. As adaboost combines multiple weak learners, the goal of user shifts from finding the best algorithm to finding an algorithm that performs better than chance (0.5). As adaboost is built upon weaker learners, it inherits all the advantages of the underlying weak learner. For example, adaboost algorithm with decision trees as weak learners inherits all the advantages of it, and alleviates some of the disadvantages of the decision trees. As the structure of decision trees are sensitive to small variations in data, combining multiple of them gives a model that is insensitive to the structure of the trees. Similarly, adaboost or other ensemble methods have the potential to take into account multiple features (via different decision trees) to alleviate the issue that decision trees make splits based on one variable alone. As adaboost combines multiple weak learners, it is in general less prone to overfitting. Further, the decision boundaries need not be parallel to individual features' axes. 
 - Disadvantage: Adaboost combines multiple weak learners, therefore a poor weak learner can adversely affect its performance. If the underlying weak learner is too complex, then there is a chance of overfitting, and if its too weak, underfitting. 
 - Why use this model: Adaboost technique combines multiple weak learners to give a strong model. In this project, adaboost is used with decision trees. This assures that the underlying models are less prone to overfitting, are not adversely affected by nonlinearities or skewed distributions. In addition using adaboost in conjunction with decision trees reduces sensitivity of the predictor model to data, and by combining different trees allows for multiple correlated features to used to make splits.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [16]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target, print_opt = True):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    if print_opt:
        print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [17]:
def gen_train_samples(n_train,X_all,y_all,seed = 42):

    # Set the number of testing points
    num_test = X_all.shape[0] - n_train
    ratio_test_tot  = float(num_test)/float(X_all.shape[0])
    sss = StratifiedShuffleSplit(y_all, 1, test_size=ratio_test_tot, random_state=seed)
    for train_index, test_index in sss:
        train_index = train_index
        test_index =  test_index
    
    # TODO: Shuffle and split the dataset into the number of training and testing points above
    X_train = X_all.ix[train_index].reset_index()
    X_train = X_train.drop('index',axis=1)
    X_test = X_all.ix[test_index].reset_index()
    X_test = X_test.drop('index',axis=1)
    y_train = y_all[train_index]
    y_test = y_all[test_index]
    return X_train,y_train,X_test,y_test

In [18]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# TODO: Initialize the three models
clf_A = LogisticRegression(random_state=66)
clf_B = DecisionTreeClassifier(random_state=0)
clf_C = AdaBoostClassifier(DecisionTreeClassifier(random_state=0),
                         algorithm="SAMME",random_state=32)

# TODO: Set up the training set sizes
# I wrote a function, gen_train_samples instead to generate training and validation sets. 

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# TODO: Execute the 'train_predict' function for each classifier and each training set size
clf_models = [clf_A,clf_B,clf_C]

for clf in clf_models:
    for n_train in [100,200,300]:
        X_train_n,y_train_n,X_test_n,y_test_n = gen_train_samples(n_train,X_all,y_all)
        train_predict(clf, X_train_n,y_train_n,X_test_n,y_test_n)
        print ' '

    print ' '
    print ' '

Training a LogisticRegression using a training set size of 100. . .
Trained model in 0.0012 seconds
Made predictions in 0.0002 seconds.
F1 score for training set: 0.9078.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.7700.
 
Training a LogisticRegression using a training set size of 200. . .
Trained model in 0.0027 seconds
Made predictions in 0.0002 seconds.
F1 score for training set: 0.8443.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.7643.
 
Training a LogisticRegression using a training set size of 300. . .
Trained model in 0.0038 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.8512.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7500.
 
 
 
Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0007 seconds
Made predictions in 0.0002 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7332.
 
Training a DecisionTreeClassifie

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Logistic regression **  



| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.0012 s           |    0.0003 s            |   0.9078         |   0.7700      |
| 200               |      0.0027 s           |    0.0002 s            |   0.8443         |   0.7643        |
| 300               |      0.0038 s           |    0.0003 s            |   0.8512         |   0.7500        |


** Decision tree **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0009 s            |     0.0001 s           |     1.0000       |    0.7332       |
| 200               |     0.0014 s            |     0.0001 s           |     1.0000       |    0.7463       |
| 300               |     0.0002 s            |     0.0001 s           |     1.0000       |    0.6557       |

 
** Adaboost **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0004 s      |     0.0005 s         |            1.0000.      |     0.7317    |
| 200               |        0.0004 s      |     0.0004 s           |             1.0000      |   0.7509        |
| 300               |        0.0004 s      |     0.0004 s           |             1.0000     |   0.7059      |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

Three models (logistic regression, decision tree and adaboost) were tested. Training data points were split into three training and validation sets such that the number of training points in the training set were 100, 200 and 300, respectively. Tables above present training times, prediction times, f1-score during training and f1-score during testing. F1-score is the harmonic mean of precision and recall, and it combines both precision and recall into a single metric. Maximizing F1-scores typically gives higher precision and recall. Comparing across models, F1-scores on training set were highest for the decision tree model and adaboost, followed by the logistic regression. These scores however did not transfer to the testing set. On testing data, logistic regression performed the best followed by adaboost and decision trees. This indicates that the decision trees overfit the data, and proper pruning is required. Adaboost showed better performance than decision trees. This is not surprising given adaboost's tendency to not overfit data and provide stable predictions. Finally logistic regression showed the best performance, however, logistic regression scores cannot be further improved without introducing regularization terms. Therefore, logistic regression was not chosen. For all the models, transfer from training to testing set was the best when 200 points were taken for training. Transfer or generalization was poor when models were trained on 100 or 300 points. Too few training points result in poor coverage and too many training points can result in test data being too sparse. 

Based on these results I chose the adaboost algorithm. Adaboost algroithm is slightly more complex than decision tree model. However, it removes several decision trees' disadvantages like overfitting, high sensitivity to data, tendency to use only one feature to perform splitting and ignoring correlated variables, and large variation in structure of trees. Although, adaboost model may be slower when more estimators are involved, the advantages offered by adaboost overweigh potentially longer training time. Further, the predictions times are expected to be in the order of milliseconds. So having longer training time for better performance is a prudent choice.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

AdaBoost or adaptive boosting is applied in conjunction with decision trees. Adaboost weighs multiple weak learners (decision trees), and combines their predictions to give an enseamble prediction. Decision tree learning partitions the predictor variables into different regions and makes predictions for each class. Advantage of decision trees is that they make no underlying assumption about the data and provide intuitive interprations. However, a poorly designed decision tree is prone to not generalizing to new data, not considering multiple correlated variables and high variance in tree structures. These issues can be alleviated by using an adaboost model on top of decision trees.

For training the model, first the data was split into 300 training data set and 95 test data. Two parameters of decision trees (maximum depth of a tree and minimum number of samples to make decision at each node) and two parameters of adaboost (number of estimators and learning rate) were varied. To keep track of training, I sampled 1 value of maximum depth, minimum number of samples and number of estimators from a discrete distribution of integers. I sampled 1 value in the initialy for the learning rate from a uniform distribution, and sampled more values as training progressed. The main idea was to throw a wider and conduct search over larger range, and restrict the range as training progresses. The training algorithm samples these parameters, and keeps track of the parameters that give the best performance. As training progresses, algorithm seeks samples parameters from a narrower range around the best set of parameters. Iterative parameter search was performed instead of gridsearch over a large grid because performing training in an iterative manner allows to monitor the training process and make any corrections if needed. Further [random search usually gives better performance](http://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf) for the same number of training sets when compared to grid search. This process is repeated until the performance on the test data converges. Once the best model is obtained, predictions are made by first making predictions from individual decision trees, and then using adaboost to combine all predictions. 

To further improve performance and stability of predicitons, all good model candidates (defined by F1-score above 0.81) are used in a final voting classifier. This step usually gives 1-2% boost in performance. 

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [12]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.metrics import make_scorer,f1_score
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline

f1_scorer = make_scorer(f1_score, greater_is_better = True, 
                        pos_label='yes')

test_best = -100
clf_all = []
F1_all  = []

min_max_depth = 9
max_max_depth = 11

min_m_split= 5
max_m_split = 7

min_n_estimtrs =  80
max_n_estimtrs = 90

min_lr = .79
max_lr = .81
num_lr = 4

for i_test in range(500):
    t1 = time()

    param_grid = {'base_estimator__max_depth' : \
                  np.random.randint(min_max_depth,high=max_max_depth, size=1),
              'base_estimator__min_samples_split' : \
                  np.random.randint(min_m_split,high=max_m_split, size=1),
              'base_estimator__splitter' :   ["best"],
              'n_estimators': \
                  np.random.randint(min_n_estimtrs,high=max_n_estimtrs, size=1),
              'learning_rate': \
                  min_lr+(max_lr - min_lr)*np.random.rand(num_lr)
             }


    DTC = DecisionTreeClassifier(max_features = "auto", 
                                 class_weight = "balanced",
                                 random_state=42)
    # TODO: Initialize the classifier
    
    clf_adb = AdaBoostClassifier(base_estimator = DTC,random_state=22)


    sss = StratifiedShuffleSplit(y_train,5, test_size=0.2, random_state=64)

    # TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
    grid_obj = GridSearchCV(clf_adb, param_grid,
                       scoring=f1_scorer,cv=sss)

    # TODO: Fit the grid search object to the training data and find the optimal parameters
    grid_obj = grid_obj.fit(X_train, y_train)
    
    
    if ((i_test>5000) & ((i_test % 400)==0)):
        min_m_split = best_min_split-round((best_min_split-min_m_split)*.8)
        max_m_split = best_min_split+round((max_m_split-best_min_split)*.8)
        
        print best_max_depth
        
        min_max_depth = best_max_depth-round((best_max_depth-min_max_depth)*.8)
        max_max_depth = best_max_depth+round((max_max_depth-best_max_depth)*.8)

        min_n_estimtrs = best_n_estmtr-round((best_n_estmtr-min_n_estimtrs)*.8)
        max_n_estimtrs = best_n_estmtr+round((max_n_estimtrs-best_n_estmtr)*.8)
        
        min_lr = best_lr-((best_lr-min_lr)*.8)
        max_lr = best_lr+((max_lr-best_lr)*.8)
        num_lr += 1
        print 'New Param search'
        print 'lr:',min_lr,max_lr
        print 'n_estimators:',min_n_estimtrs,max_n_estimtrs
        print 'max_depths:',min_max_depth,max_max_depth
        print 'min_splits:',min_m_split,max_m_split
    # Get the estimator
    clf = grid_obj.best_estimator_
       
    test_mod = predict_labels(clf, X_test, y_test,False)
    t2 = time()-t1
    
    if (i_test % 40000)==0:
        print('Status: Loop # %d .'%(i_test))
    
    if (test_mod>0.82):
        clf_all.append(clf)
        F1_all.append(test_mod)
        print('Good: Loop # %d took %0.4f seconds. F1-test = %0.5f, F1_loop = %0.5f.'\
              %(i_test,t2,test_best,test_mod))
        
    if (test_best<test_mod):
        #print i_test
        clf_best = clf
        test_best = test_mod
        train_best = predict_labels(clf_best, X_train, y_train,False)
        best_min_split = clf_best.base_estimator.min_samples_split
        best_max_depth = clf_best.base_estimator.max_depth
        best_n_estmtr = clf_best.n_estimators
        best_lr = clf_best.learning_rate
        print('Best: Loop # %d took %0.4f s. F1-train = %0.4f,F1-test = %0.4f, F1_loop = %0.4f.'\
              %(i_test,t2,train_best,test_best,test_mod))
        #print clf
        
#Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf_best,
                                                                             X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf_best,
                                                                            X_test, y_test))

Status: Loop # 0 .
Best: Loop # 0 took 5.2541 s. F1-train = 1.0000,F1-test = 0.7703, F1_loop = 0.7703.
Best: Loop # 1 took 4.9416 s. F1-train = 1.0000,F1-test = 0.7862, F1_loop = 0.7862.
Best: Loop # 3 took 5.3476 s. F1-train = 1.0000,F1-test = 0.7973, F1_loop = 0.7973.
Best: Loop # 6 took 5.3979 s. F1-train = 1.0000,F1-test = 0.8000, F1_loop = 0.8000.
Best: Loop # 31 took 5.7275 s. F1-train = 1.0000,F1-test = 0.8056, F1_loop = 0.8056.
Best: Loop # 94 took 5.1748 s. F1-train = 1.0000,F1-test = 0.8085, F1_loop = 0.8085.
Best: Loop # 190 took 5.6408 s. F1-train = 1.0000,F1-test = 0.8108, F1_loop = 0.8108.
Best: Loop # 245 took 5.0600 s. F1-train = 1.0000,F1-test = 0.8163, F1_loop = 0.8163.
Best: Loop # 305 took 5.5893 s. F1-train = 1.0000,F1-test = 0.8188, F1_loop = 0.8188.
Good: Loop # 349 took 5.1649 seconds. F1-test = 0.81879, F1_loop = 0.82192.
Best: Loop # 349 took 5.1649 s. F1-train = 1.0000,F1-test = 0.8219, F1_loop = 0.8219.
Good: Loop # 455 took 4.9590 seconds. F1-test = 0.82192

In [26]:
#print clf_best
print ''
print '------Best model, Adaboost+DecisionTrees-----'
print 'Best f1-score obtained for,'
print('Learning Rate: %0.2f' % clf_best.learning_rate)
print('Number of estimators: %0.2f' % clf_best.n_estimators)
print('Max depth: %0.2f' % clf_best.base_estimator.max_depth)
print('Min samples to split: %0.2f' % clf_best.base_estimator.min_samples_split)
print('Splitter: Best')
print ''
print('F1-score for the best model: %0.2f'%test_best)


------Best model-----
Best f1-score obtained for,
Learning Rate: 0.81
Number of estimators: 81.00
Max depth: 10.00
Min samples to split: 5.00
Splitter: Best

F1-score for the best model: 0.84


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

The final F1-score for the tuned model was \\( 0.84 \\) compared to \\(0.7059\\) for the untuned model. By performing parameter tuning using grid-search and iterative random search, the overall f1-score increased by \\( 19 \% \\). 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.